# Collecting temperature data from an API

## About the data
In this notebook, we will be collecting daily temperature data from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). We will use the Global Historical Climatology Network - Daily (GHCND) dataset; see the documentation [here](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

## Using the NCEI API
 [here](https://www.ncdc.noaa.gov/cdo-web/token) 에서 토큰 요청

In [36]:
import requests

def make_request(endpoint, payload=None):
    """
    헤더와 선택적 페이로드를 전달하는 기상 API의 특정 종단점에 요청한다.
    
    매개변수(Parameters):
        -종단점(endpoint) :GET 요청(request)을 하려는 API의 종단점
        -페이로드(payload): 요청과 함께 전달할 데이터의 딕셔너리
        
    반환값(Returns):
        응답(response) 객체
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'rmGGNDuTrUhcSoddJhzlmXctMUBcEqzK'
        },
        params=payload
    )

**Note: API는 초당 5개의 요청과 1일 1만건의 요청으로 제한된다..**

제한을 초과하면 상태코드가 클라이언트 에러 표시(400번대)

-404요청 받은 자원을 찾을 수 없다.

-400서버가 요청을 이해할 수 앖가니 요청 처리를 거부했다는 것

서버의 에러상태 표시(500번대)

## See which datasets are available
다음과 같은 코드로 2018년 10월 1일부터 오늘 날짜까지의 범위의 데이터셋을 확인할 수 있다.

In [37]:
response = make_request('datasets', {'startdate': '2018-10-01'})

요청이 성공했는지 확인하려면 status_code 속성 확인

In [38]:
response.status_code

200

`ok` 속성으로도 확인 가능

In [39]:
response.ok

True

### Get the keys of the result
응답을 받으면 json()메서드를 이용해 페이로드를 얻을 수 있다.

그 후, 딕셔너리 메서드를 사용해보고 싶은 부분을 선택할 수 있다.

In [40]:
payload = response.json()
payload.keys()

dict_keys(['metadata', 'results'])

**metadata**는 결과데이터에 관한 정보

**results**는 실제 결과 데이터가 들어 있다.

In [41]:
payload['metadata']

{'resultset': {'offset': 1, 'count': 11, 'limit': 25}}

-> 결과 데이터에 **11개의 행**이 있는 걸 알 수 있다. **(count: 11)**

### Figure out what data is in the result 
`results` 키를 확인해서 어떤 필드가 있는지 볼 수 있다.

In [42]:
payload['results'][0].keys()

dict_keys(['uid', 'mindate', 'maxdate', 'name', 'datacoverage', 'id'])

### Parse the result
모든 필드가 필요한 것이 아니므로 리스트 컴프리헨션을 적용해 

**`id`와 `name` 만 볼 수 있도록** 한다.

In [43]:
[(data['id'], data['name']) for data in payload['results']]

[('GHCND', 'Daily Summaries'),
 ('GSOM', 'Global Summary of the Month'),
 ('GSOY', 'Global Summary of the Year'),
 ('NEXRAD2', 'Weather Radar (Level II)'),
 ('NEXRAD3', 'Weather Radar (Level III)'),
 ('NORMAL_ANN', 'Normals Annual/Seasonal'),
 ('NORMAL_DLY', 'Normals Daily'),
 ('NORMAL_HLY', 'Normals Hourly'),
 ('NORMAL_MLY', 'Normals Monthly'),
 ('PRECIP_15', 'Precipitation 15 Minute'),
 ('PRECIP_HLY', 'Precipitation Hourly')]

## Figure out which data category we want

결과의 첫번째 항목이 우리가 필요한 데이터이다. 이제 datasetid에 대한 값 GHCND을 얻었으므로 기온 데이터 요청에 필요한 datacategoryid에 대한 값을 식별해야 한다. 이를 위해 datacategories 종단점을 사용한다. 

여기서는 JSON 페이로드가 그리 크지 않으므로 JSON페이로드를 인쇄할 수 있다.

In [44]:
# get data category id
response = make_request(
    'datacategories', payload={'datasetid': 'GHCND'}
)
response.status_code

200

reponse.join()는 **dict_keys(['metadata', 'results'])** 이러한 형태이고,

우리는 `results` 부분에 무엇이 들어가 있는지 확인한다.

In [45]:
response.json()['results']

[{'name': 'Evaporation', 'id': 'EVAP'},
 {'name': 'Land', 'id': 'LAND'},
 {'name': 'Precipitation', 'id': 'PRCP'},
 {'name': 'Sky cover & clouds', 'id': 'SKY'},
 {'name': 'Sunshine', 'id': 'SUN'},
 {'name': 'Air Temperature', 'id': 'TEMP'},
 {'name': 'Water', 'id': 'WATER'},
 {'name': 'Wind', 'id': 'WIND'},
 {'name': 'Weather Type', 'id': 'WXTYPE'}]

## Grab the data type ID for the temperature category
우리는 기온 데이터가 필요하므로 `TEMP` 데이터 카테고리를 원한다.

datatypes를 종단점으로 사용해 데이터 유형을 식별한다.

`datacategoryid` 는`TEMP` 로 페이로드를 지정하고, 페이로드와 함께 반환될 한계도 지정한다.

In [46]:
# get data type id
response = make_request(
    'datatypes',
    payload={
        'datacategoryid': 'TEMP', 
        'limit': 100
    }
)
response.status_code

200

`id` 와 `name` 필드만 받아 온다. TAVG, TMAX, TMIN이 해당되는 마지막 5개의 데이터만 가져온다. 

In [47]:
[(datatype['id'], datatype['name']) for datatype in response.json()['results']][-5:] # look at the last 5

[('MNTM', 'Monthly mean temperature'),
 ('TAVG', 'Average Temperature.'),
 ('TMAX', 'Maximum temperature'),
 ('TMIN', 'Minimum temperature'),
 ('TOBS', 'Temperature at the time of observation')]

## Determine which location category we want

.이번엔 `locationcategories` 종단점을 사용해 TAVG, TMAX, TMIN으로 데이터 범위를 좁힌다. 

datasetid는 GHCND로 페이로드를 지정한다.

In [48]:
# get location category id 
response = make_request(
    'locationcategories', 
    payload={'datasetid': 'GHCND'}
)
response.status_code

200

In [49]:
import pprint
pprint.pprint(response.json())

{'metadata': {'resultset': {'count': 12, 'limit': 25, 'offset': 1}},
 'results': [{'id': 'CITY', 'name': 'City'},
             {'id': 'CLIM_DIV', 'name': 'Climate Division'},
             {'id': 'CLIM_REG', 'name': 'Climate Region'},
             {'id': 'CNTRY', 'name': 'Country'},
             {'id': 'CNTY', 'name': 'County'},
             {'id': 'HYD_ACC', 'name': 'Hydrologic Accounting Unit'},
             {'id': 'HYD_CAT', 'name': 'Hydrologic Cataloging Unit'},
             {'id': 'HYD_REG', 'name': 'Hydrologic Region'},
             {'id': 'HYD_SUB', 'name': 'Hydrologic Subregion'},
             {'id': 'ST', 'name': 'State'},
             {'id': 'US_TERR', 'name': 'US Territory'},
             {'id': 'ZIP', 'name': 'Zip Code'}]}


## Get NYC Location ID
뉴욕시에 대한 locationid를 찾기 위해 우리는 모든 도시들을 검색할 수 있어야 한다. 정렬된 도시들을 API에 요청할 수 있으므로 우리는 많은 요청을 보내지 않고, **이진 검색**으로 빠르게 뉴욕시에 대한 데이터를 찾을 수 있다. 

In [50]:
def get_item(name, what, endpoint, start=1, end=None):
    """
    이진 검색으로 JSON 페이로드 가져오기

    파라미터:
        - name: 찾고 있는 항목.
        - what: 'name' 항목이 무엇인지 지정하는 딕셔너리.
        - endpoint: 항목을 찾을 위치
        - start: 시작위치. 이 값을 수정할 필요는 없지만 함수는 재귀적으로 이 값을 조작한다.
        - end: 항목의 마지막 위치. 중간점을 찾는 데 사용되지만 'start' 처럼 신경쓰지 않아도 된다.

    반환값: 항목을 찾았다면 항목 정보의 딕셔너리, 찾지 못했다면 빈 딕셔너리.
    """
    # 매번 데이터를 절반으로 자르기 위한 중간점을 찾는다.
    mid = (start + (end or 1)) // 2
    
    # 대소문자를 구별하지 않도록 소문자로 변환한다.
    name = name.lower()
    
    # 각 요청에서 전송할 페이로드를 정의한다.
    payload = {
        'datasetid': 'GHCND',
        'sortfield': 'name',
        'offset': mid, # 매번 offset을 바꾼다.
        'limit': 1 # 1개의 값만 받는다.
    }
    
    # 'what'에 추가 필터를 추가하도록 요청한다.
    response = make_request(endpoint, {**payload, **what})
    
    if response.ok:
        payload = response.json()

        # 응답이 ok인 경우 응답 메타데이터에서 마지막 인덱스를 가져온다.
        end = end or payload['metadata']['resultset']['count']
        
        # 현재 이름의 소문자 버전을 가져온다.
        current_name = payload['results'][0]['name'].lower()
        
        # 검색하고 있는 것이 현재 이름에 있다면 항목을 찾은 것이다.
        if name in current_name:
            return payload['results'][0] # 찾은 항목을 반환한다.
        else:
            if start >= end: 
                # 시작 인덱스가 마지막 인덱스보다 크거나 같으면 항목을 찾을 수 없다.
                return {}
            elif name < current_name:
                # 이름이 알파벳 순으로 현재 이름보다 앞에 있으면, 데이터 절반의 왼쪽에서 검색한다.
                return get_item(name, what, endpoint, start, mid - 1)
            elif name > current_name:
                # 이름이 알파벳 순으로 현재 이름보다 후에 있으면 데이터 절반의 오른쪽에서 검색한다.
                return get_item(name, what, endpoint, mid + 1, end)    
    else:
        # 응답이 ok가 아니면 그 이유를 알고자 상태 코드를 출력한다.
        print(f'Response not OK, status: {response.status_code}')

위 코드는 알고리즈의 재귀적 구현으로 함수 내주에서 함수 자체를 다시 호출한다는 것을 뜻한다. 재귀함수를 구현할 때는 함수가 무한루프에 빠지지 않고 멈출 수 있도록 기본 조건을 잘 정의해야 한다. 

우리가 이진검색을 사용해 뉴욕시를 찾을 때, 1983개의 항목의 중앙 근처에 있음에도, 8개의 요청만으로 찾을 수 있다.

In [51]:
# get NYC id 
nyc = get_item('New York', {'locationcategoryid': 'CITY'}, 'locations')
nyc

{'mindate': '1869-01-01',
 'maxdate': '2023-09-17',
 'name': 'New York, NY US',
 'datacoverage': 1,
 'id': 'CITY:US360019'}

## Get the station ID for Central Park
다시 이진 검색을 사용해 Central Park의 관측소의 관측소ID를 얻을 수 있다.

In [52]:
central_park = get_item('NY City Central Park', {'locationid': nyc['id']}, 'stations')
central_park

{'elevation': 42.7,
 'mindate': '1869-01-01',
 'maxdate': '2023-09-16',
 'latitude': 40.77898,
 'name': 'NY CITY CENTRAL PARK, NY US',
 'datacoverage': 1,
 'id': 'GHCND:USW00094728',
 'elevationUnit': 'METERS',
 'longitude': -73.96925}

## Request the temperature data
최종적으로 2018년 10월 센트럴 파크에서 기록된 뉴욕시의 섭씨 기온을 요청.

API탐색을 통해 수집한 모든 매개변수 사용.

In [53]:
# get NYC daily summaries data 
response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': central_park['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # average, max, and min temperature
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

200

그리고 API와 관련해 JSON 페이로드가 널리 사용되므로 JSON에 익숙해지는 것이 좋다.

## Create a DataFrame
이제 DataFrame객체로 만든다.

In [54]:
import pandas as pd

df = pd.DataFrame(response.json()['results'])
df.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",24.4
1,2018-10-01T00:00:00,TMIN,GHCND:USW00094728,",,W,2400",17.2
2,2018-10-02T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",25.0
3,2018-10-02T00:00:00,TMIN,GHCND:USW00094728,",,W,2400",18.3
4,2018-10-03T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",23.3


`TAVG` 에 대한 데이터는 얻지 못했다. 해당 관측소에서 기록하지 않는다.

In [55]:
df.datatype.unique()

array(['TMAX', 'TMIN'], dtype=object)

In [56]:
if get_item(
    'NY City Central Park', {'locationid': nyc['id'], 'datatypeid': 'TAVG'}, 'stations'
):
    print('Found!')

Found!


## Using a different station
`TAVG`속성이 있는 **뉴욕시의 라과디아 공항 관측소** 데이터를 사용해보자.

In [57]:
laguardia = get_item(
    'LaGuardia', {'locationid': nyc['id']}, 'stations'
)
laguardia

{'elevation': 3,
 'mindate': '1939-10-07',
 'maxdate': '2023-09-17',
 'latitude': 40.77945,
 'name': 'LAGUARDIA AIRPORT, NY US',
 'datacoverage': 1,
 'id': 'GHCND:USW00014732',
 'elevationUnit': 'METERS',
 'longitude': -73.88027}

In [58]:
# get NYC daily summaries data 
response = make_request(
    'data', 
    {
        'datasetid': 'GHCND',
        'stationid': laguardia['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # temperature at time of observation, min, and max
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

200

In [59]:
df = pd.DataFrame(response.json()['results'])
df.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",21.2
1,2018-10-01T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",25.6
2,2018-10-01T00:00:00,TMIN,GHCND:USW00014732,",,W,2400",18.3
3,2018-10-02T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",22.7
4,2018-10-02T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",26.1


In [60]:
df.datatype.value_counts()

TAVG    31
TMAX    31
TMIN    31
Name: datatype, dtype: int64

In [26]:
df.to_csv('data/nyc_temperatures.csv', index=False)

<hr>
<div>
    <a href="./1-wide_vs_long.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./3-cleaning_data.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>